In [116]:
import pandas as pd

In [117]:
with open('./ss_field/ss_north_field_env_data.txt.txt') as f:
    lines = [line.rstrip() for line in f]

In [154]:
df=pd.DataFrame()

In [155]:
for line in lines:
    s=line.split(',')
    dic = {}
    dic['Date']=s[0][0:19]
    for i in range(1,len(s)):
        a=s[i].split(':')
        a[0]=a[0].rstrip()
        a[0]=a[0].lstrip()
        a[1]=a[1].rstrip()
        a[1]=a[1].lstrip()
        dic[a[0]]=a[1]
    if int(dic['Intensity']) > 100:
        df=df.append(dic,ignore_index=True)

In [156]:
df.head()

,BME280_Humidity,BME280_Pressure,BME280_Temp,DS18B20_Temp,Date,Intensity
0,66.70,900.24,15.74,10.06,2019-10-26-07-25-58,155
1,66.72,900.29,15.74,10.88,2019-10-26-07-25-59,215
2,66.55,900.40,16.02,10.94,2019-10-26-07-41-07,215
3,66.10,900.59,16.60,11.44,2019-10-26-07-56-14,280
4,65.65,900.51,17.04,12.25,2019-10-26-08-11-21,431


In [157]:
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d-%H-%M-%S')
df['BME280_Humidity']=df['BME280_Humidity'].astype(float)
df['BME280_Pressure']=df['BME280_Pressure'].astype(float)
df['DS18B20_Temp']=df['DS18B20_Temp'].astype(float)
df['Intensity']=df['Intensity'].astype(float)
df['BME280_Temp']=df['BME280_Temp'].astype(float)

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BME280_Humidity  983 non-null    float64       
 1   BME280_Pressure  983 non-null    float64       
 2   BME280_Temp      983 non-null    float64       
 3   DS18B20_Temp     983 non-null    float64       
 4   Date             983 non-null    datetime64[ns]
 5   Intensity        983 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 46.2 KB


In [160]:
import glob
import numpy as np
from datetime import datetime
fnames=glob.glob("./ss_field/images/*.jpg")
fnames.sort()

In [202]:
fnames[0]

'./ss_field/images/image2019-10-26-07-25-59.jpg'

In [167]:
d=df.Date
d=np.array(d)

In [197]:
ds=[]
for i in range(len(d)):
    s=str(d[i]).split('T')
    x=list(s[0]+"-"+s[1][0:8])
    x[13]='-'
    x[16]='-'
    ds.append(''.join(x))
l=np.array([0 for i in range(983)])
fx=[]
for filename in fnames:
    s=filename.lstrip('./ss_field/images/image').rstrip('.jpg')[0:19]
    fx.append(s)

In [201]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray


In [203]:
for idx,filedate in enumerate(ds): 
    if filedate in fx:
        filename='./ss_field/images/image'+filedate+'.jpg'
        cnt=0
        image=plt.imread(filename)
        gray=rgb2gray(image)
        gray=gray[650:2163, 1504:2608]
        grayx=gray.reshape(gray.shape[0]*gray.shape[1])
        m=grayx.mean() + 0.1
        for i in range(gray.shape[0]):
            for j in range(gray.shape[1]):
                if gray[i][j] > m:
                    gray[i][j]=1
                    cnt=cnt+1
                else :
                    gray[i][j]=0

        l[idx]=cnt

In [205]:
df['Pixels']=l

In [233]:
df

,BME280_Humidity,BME280_Pressure,BME280_Temp,DS18B20_Temp,Date,Intensity,Pixels
1,66.72,900.29,15.74,10.88,2019-10-26 07:25:59,215.0,95667
2,66.55,900.40,16.02,10.94,2019-10-26 07:41:07,215.0,90264
3,66.10,900.59,16.60,11.44,2019-10-26 07:56:14,280.0,85278
4,65.65,900.51,17.04,12.25,2019-10-26 08:11:21,431.0,84546
5,64.49,900.51,18.28,13.12,2019-10-26 08:26:28,700.0,80045
...,...,...,...,...,...,...,...
975,43.52,898.74,27.11,18.12,2019-11-20 12:43:22,2081.0,242312
976,44.26,898.45,26.04,18.00,2019-11-20 12:58:36,1664.0,238353
977,43.79,898.09,26.86,19.44,2019-11-20 13:13:49,1555.0,235785
978,43.42,897.88,27.45,19.25,2019-11-20 13:29:03,1863.0,239238


In [214]:
index=df[df.Pixels==0].index

In [219]:
index


Int64Index([  0,  11,  40,  51,  80,  88, 115, 157, 194, 198, 231, 236, 240,
            268, 281, 309, 321, 357, 358, 397, 437, 461, 467, 475, 500, 506,
            511, 514, 546, 558, 583, 622, 661, 664, 700, 707, 712, 734, 766,
            772, 794, 797, 807, 808, 832, 843, 856, 863, 878, 888, 916, 918,
            954, 980, 981, 982],
           dtype='int64')

In [226]:
df=df.drop(index)

In [230]:
df.to_csv('Weather_vs_Pixel_Area.csv')

,BME280_Humidity,BME280_Pressure,BME280_Temp,DS18B20_Temp,Date,Intensity,Pixels
1,66.72,900.29,15.74,10.88,2019-10-26 07:25:59,215.0,95667
2,66.55,900.40,16.02,10.94,2019-10-26 07:41:07,215.0,90264
3,66.10,900.59,16.60,11.44,2019-10-26 07:56:14,280.0,85278
4,65.65,900.51,17.04,12.25,2019-10-26 08:11:21,431.0,84546
5,64.49,900.51,18.28,13.12,2019-10-26 08:26:28,700.0,80045
...,...,...,...,...,...,...,...
975,43.52,898.74,27.11,18.12,2019-11-20 12:43:22,2081.0,242312
976,44.26,898.45,26.04,18.00,2019-11-20 12:58:36,1664.0,238353
977,43.79,898.09,26.86,19.44,2019-11-20 13:13:49,1555.0,235785
978,43.42,897.88,27.45,19.25,2019-11-20 13:29:03,1863.0,239238
